<a href="https://colab.research.google.com/github/Nuthan03/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intorduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language(could be seqeuences of texts or speech).
Another common term for NLP problems is sequence to sequence problems(seq2seq)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5631eaa4-5a89-d52d-2ae8-807437f86d80)


## Get the helper functions


In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-15 08:29:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-15 08:29:08 (92.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we are going to be using us Kaggle's introduction to NLP datasets (text samples of Tweets labelled as diaster or not diaster).

In [3]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-15 08:29:12--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-12-15 08:29:12 (143 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/


In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real diaster)
Text:
If it wasn't for the Blood! ????

---

Target: 1 (real diaster)
Text:
70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/pmS4pMuR0q

---

Target: 0 (not real diaster)
Text:
#PBBan (Temporary:300) Russaky89 @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason

---

Target: 0 (not real diaster)
Text:
WCE I can't even lie even tho I can't stand her she still will always be my crush ?? @_explicitpretty

---

Target: 0 (not real diaster)
Text:
http://t.co/EQjCpWILVn: Articles In Saudi Press Reject Russian Initiative For Regional Alliance With Assad Regime To Fight Terrorism

---



## Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this namely:
* Tokenziation-direct mapping of token(a token could be a word or a character) to number
* Embedding-create a martrix of feature vector for each token(the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization(tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabullary
                                    standardize="lower_and_strip_punctuation", # How to process text
                                    split="whitespace", # How to split tokens
                                    ngrams=None, # Grouping words is none
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # How long should the output sequence of tokens be?
                                    #pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text vectorization wtth custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 'My Fifty Online Dates and why I'm still single' by Michael Windstorm $2.99 Amazon http://t.co/5KohO39oJE    #kindle  #datingtips #goodreads        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  13, 5645,  812, 5860,    7,   91,   32,   80, 1366,   18, 1273,
         428, 6332, 1329,    1]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we are going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:


*   `input_dim` = the size of our vocabulary.
*   `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
*   `input_length` = length of the sequences being passed to the embedding layer



In [22]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform",
                             input_length=max_length, # how long is each input
                             name="embedding")

embedding


In [23]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
       \n\nEmbedded version")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@Stretcher @witter @Rexyy @Towel show me a picture of it       

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00862866, -0.0484234 ,  0.04515988, ..., -0.04846802,
         -0.03857199,  0.03199461],
        [ 0.03120024, -0.0211521 , -0.02284262, ...,  0.03351292,
         -0.01334592,  0.00214299],
        [-0.0287181 , -0.02497869, -0.03439988, ...,  0.00718404,
         -0.04803692, -0.04604557],
        ...,
        [-0.00649364,  0.02961625,  0.04447189, ..., -0.03951361,
         -0.0058566 , -0.01570613],
        [-0.00649364,  0.02961625,  0.04447189, ..., -0.03951361,
         -0.0058566 , -0.01570613],
        [-0.00649364,  0.02961625,  0.04447189, ..., -0.03951361,
         -0.0058566 , -0.01570613]]], dtype=float32)>

In [24]:
# Chcek out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00862866, -0.0484234 ,  0.04515988, -0.00228381, -0.01662446,
        -0.00042621,  0.03832686, -0.007548  , -0.0128627 , -0.04274279,
        -0.00638896, -0.00516098,  0.01046723,  0.01569715, -0.01459292,
         0.01955612,  0.01726046,  0.01139789, -0.00181284, -0.0110295 ,
         0.04518031, -0.0410716 , -0.03199041,  0.0133431 ,  0.03265803,
        -0.00383271,  0.0048233 , -0.03594848,  0.01584219,  0.00581412,
         0.00636306, -0.02197566,  0.01732175,  0.04149613,  0.03472935,
        -0.01022085,  0.01319922, -0.01882542,  0.04715406,  0.04719872,
         0.03527785, -0.00905763,  0.00658699,  0.03005096,  0.00733759,
         0.02504145, -0.01440893,  0.01776481,  0.04151158, -0.00238479,
        -0.03974231, -0.03007454, -0.01420742,  0.01260707, -0.01876588,
        -0.04087013, -0.0083034 , -0.01193553,  0.00882922, -0.02679348,
        -0.03037138, -0.04346257, -0.02414446, -0.03350627,  0.03229157,
  

## Modelling a text dataset

Now we have got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Navie Bayes(baseline), this is from Sklearn ML map.
* Model 1: Feed-forward neural network(dense model)
* Model 2: LSTM model(RNN)
* Model 3: GRU model(RNN)
* Model 4: Bidirectional-LSTM model(RNN)
* Model 5: 1D Convolutional Neural Network(CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor(using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to apporach all these 6 model?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experimentsto build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

>`⛏Note:` It's common pratice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numebr using tfidf
    ("clif", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clif', MultinomialNB())])

In [26]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achives an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

we could evaluate these as they are but since we are going to be evaluating several models in the same way going forward, let's create a helper function which takes an array of predictions and ground truth labels and computes the following:

 * Accuracy
 * Precision
 * Recall
 * F1-score
> `⚓Note:`Since we are dealing with a classfication problem, the above metrics are the most appropriate. If we are working with a regression problem, other metrics such as MAE (mean absolute error) would be a better choice


In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculaute_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1-score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision , recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [29]:
# Get baseline results
baseline_results = calculaute_results(y_true=val_labels,
                                      y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

#### Creating `wandb callback` since Tendorboard will be shutdown from 2024


In [30]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [31]:
import wandb
from wandb.keras import WandbCallback
import datetime

def create_wandb_callback(dir_name, experiment_name):
  """
  Create a Wandb callback to store experiment logs.
  """
  wandb.init(project=dir_name, name=experiment_name)
  print(f"Saving Wandb logs to directory: {dir_name}, experiment: {experiment_name}")

  return WandbCallback()

In [32]:
# Create a dir to save Wandb logs
SAVE_DIR = "model_logs"

In [33]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [35]:
# Compile model_1
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [36]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_wandb_callback(SAVE_DIR,
                                                                 "simple_dense_model")])

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


...

215/215 [==============================] - 10s 39ms/step - loss: 0.6106 - accuracy: 0.6927 - val_loss: 0.5370 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - ETA: 0s - loss: 0.4420 - accuracy: 0.8192

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231215_083000-6l22off2/files/model-best)... Done. 0.0s


215/215 [==============================] - 3s 13ms/step - loss: 0.4420 - accuracy: 0.8192 - val_loss: 0.4695 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - ETA: 0s - loss: 0.3470 - accuracy: 0.8606

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231215_083000-6l22off2/files/model-best)... Done. 0.1s


215/215 [==============================] - 3s 13ms/step - loss: 0.3470 - accuracy: 0.8606 - val_loss: 0.4593 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2855 - accuracy: 0.8918 - val_loss: 0.4643 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2386 - accuracy: 0.9123 - val_loss: 0.4767 - val_accuracy: 0.7861


In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4767 - accuracy: 0.7861


[0.476728230714798, 0.7860892415046692]

In [38]:
# Make some predictions and evalaute those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
model_1_pred_probs[:10]

array([[0.4062434 ],
       [0.74685943],
       [0.9979195 ],
       [0.11057913],
       [0.10472709],
       [0.9354479 ],
       [0.9153372 ],
       [0.992728  ],
       [0.9689306 ],
       [0.26595095]], dtype=float32)

In [40]:
# Covert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# Calclate our model_1 results
model_1_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [43]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

##Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [46]:
# Get the weight matrix of embedding layer
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [47]:
embed_weights

array([[ 0.00905984,  0.01961791,  0.06110724, ..., -0.02472831,
         0.01072987, -0.03088029],
       [ 0.05016447, -0.03216435, -0.03239137, ..., -0.02789739,
        -0.01118011, -0.01084244],
       [ 0.02968678, -0.03177599,  0.03695793, ..., -0.02525108,
         0.01096632, -0.05046684],
       ...,
       [ 0.03495869, -0.04138952, -0.02414127, ...,  0.02683498,
        -0.00929235,  0.01491631],
       [ 0.036291  , -0.04976005,  0.00827496, ...,  0.0392806 ,
         0.03021491,  0.00059062],
       [ 0.10421046, -0.09283836,  0.08953352, ...,  0.04702103,
         0.06109732, -0.07131317]], dtype=float32)

Now we have got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize our embedding.

To use the Embedding vectors (same as embedding weights).

* The embedding vectors (same as embedding weights).
* The meta data of the embedding vectors(the words they represent-our vocabulary).


In [48]:
# Create embedding files (we got this from Tensorflow's word embeddings )
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# Download files from colab to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
   pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

> 📖 **Resources:**
> * [MIT Deep Learning Lecture on Recurrent Neural Networks](https://youtu.be/SEnXr6v2ifU) - explains the background of recurrent neural networks and introduces LSTMs.
> * [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy - demonstrates the power of RNN's with examples generating various sequences.
> * [Understanding LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Chris Olah - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.


## Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this

```
Input (text) -> Tokenize -> Embedding -> Layers (RNN's/dense) -> Output (label probability) ```

In [50]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x)
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [51]:
# Get model summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [52]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [53]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_wandb_callback(SAVE_DIR,
                                                               "model_2_LSTM")])

accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▆██▇
val_loss,█▂▁▁▃
accuracy,0.91228
best_epoch,2
best_val_loss,0.45931
epoch,4
loss,0.23857
val_accuracy,0.78609


Saving Wandb logs to directory: model_logs, experiment: model_2_LSTM
Epoch 1/5
215/215 [==============================] - ETA: 0s - loss: 0.2252 - accuracy: 0.9191

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231215_083023-tlqzqklw/files/model-best)... Done. 0.1s


215/215 [==============================] - 20s 64ms/step - loss: 0.2252 - accuracy: 0.9191 - val_loss: 0.5398 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1629 - accuracy: 0.9407 - val_loss: 0.6028 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1286 - accuracy: 0.9520 - val_loss: 0.7359 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1069 - accuracy: 0.9599 - val_loss: 0.8306 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0829 - accuracy: 0.9656 - val_loss: 0.9546 - val_accuracy: 0.7835


In [54]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[5.3632678e-03],
       [6.7832220e-01],
       [9.9953794e-01],
       [2.7815506e-02],
       [2.7398171e-04],
       [9.9824023e-01],
       [7.7511376e-01],
       [9.9970227e-01],
       [9.9952292e-01],
       [5.5248344e-01]], dtype=float32)

In [55]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [56]:
# Calculate model 2 results
model_2_results = calculaute_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results


{'accuracy': 78.34645669291339,
 'precision': 0.785851650078301,
 'recall': 0.7834645669291339,
 'f1': 0.7813141680786296}

## Model 3:GRU

Another popular and effective RNN component is the GRU or gated recurrent until.

The GRU cell has similar features to an LSTM cell but has less parameters.

> 📖 **Resource:** A full explanation of the GRU cell is beyond the scope of this noteook but I'd suggest the following resources to learn more:
* [Gated Recurrent Unit](https://en.wikipedia.org/wiki/Gated_recurrent_unit) Wikipedia page
* [Understanding GRU networks](https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be) by Simeon Kostadinov

To use the GRU cell in TensorFlow, we can call the [`tensorflow.keras.layers.GRU()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU) class.

The architecture of the GRU-powered model will follow the same structure we've been using:

```
Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)
```

In [57]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return_sequences=True
#x = layers.LSTM(42, return_sequences=True)(x)
#x = layers.GRU(99)(x)
#x = layers.Dense(64, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [58]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d_1  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 

In [59]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [60]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_wandb_callback(SAVE_DIR,
                                                               "model_3_GRU")])

accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▂▃█▅
val_loss,▁▂▄▆█
accuracy,0.96555
best_epoch,0
best_val_loss,0.53982
epoch,4
loss,0.08286
val_accuracy,0.78346


Saving Wandb logs to directory: model_logs, experiment: model_3_GRU
Epoch 1/5
215/215 [==============================] - ETA: 0s - loss: 0.1777 - accuracy: 0.9441

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231215_083101-k58h9owo/files/model-best)... Done. 0.1s


215/215 [==============================] - 15s 58ms/step - loss: 0.1777 - accuracy: 0.9441 - val_loss: 0.6391 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1053 - accuracy: 0.9622 - val_loss: 0.7457 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0865 - accuracy: 0.9691 - val_loss: 0.9454 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0796 - accuracy: 0.9705 - val_loss: 0.9301 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0677 - accuracy: 0.9730 - val_loss: 0.9147 - val_accuracy: 0.7559


In [61]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[0.1187929 ],
        [0.833493  ],
        [0.99992394],
        [0.0650822 ],
        [0.00184518],
        [0.99889684],
        [0.98541856],
        [0.99997425],
        [0.9999813 ],
        [0.62254673]], dtype=float32))

In [62]:
# Convert model_3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

## Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, a bidirectional RNN goes from right to left as well as left to right.

In [63]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [64]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [65]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [66]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_wandb_callback(SAVE_DIR,
                                                               "model_4_bidirectional")])

accuracy,▁▅▇▇█
epoch,▁▃▅▆█
loss,█▃▂▂▁
val_accuracy,█▇▃▆▁
val_loss,▁▃██▇
accuracy,0.973
best_epoch,0
best_val_loss,0.6391
epoch,4
loss,0.0677
val_accuracy,0.75591


Saving Wandb logs to directory: model_logs, experiment: model_4_bidirectional
Epoch 1/5
215/215 [==============================] - ETA: 0s - loss: 0.1136 - accuracy: 0.9666

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231215_083132-byosnp1q/files/model-best)... Done. 0.1s


215/215 [==============================] - 23s 87ms/step - loss: 0.1136 - accuracy: 0.9666 - val_loss: 0.8467 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0602 - accuracy: 0.9752 - val_loss: 1.0828 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0497 - accuracy: 0.9781 - val_loss: 1.3290 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0462 - accuracy: 0.9794 - val_loss: 1.1983 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0413 - accuracy: 0.9803 - val_loss: 1.5242 - val_accuracy: 0.7651


In [67]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[1.1918082e-02],
       [7.2000277e-01],
       [9.9993217e-01],
       [1.4822406e-01],
       [1.3336866e-05],
       [9.9993122e-01],
       [9.9921048e-01],
       [9.9997950e-01],
       [9.9995899e-01],
       [9.9878806e-01]], dtype=float32)

In [68]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
# Calculate the results of our bidirectional model
model_4_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_4_preds)
model_4_results

{'accuracy': 76.50918635170603,
 'precision': 0.7649784764888368,
 'recall': 0.7650918635170604,
 'f1': 0.7640606406379992}

## Convolutional Neural Network for Text (and other types of sequences)

we have used CNN's for images but images are typically 2D (height x width).. however, our text data is 1D.

Previously we've Conv2D for our image data but now we are going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layers -> Outputs (class probabilities)
```

## Model 5: Conv1D



In [72]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # also referred to as an ngram of 5 (meaning it looks at 5 words at a time)
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test) # Pass test embedding through conv1D layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [73]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.07767706, -0.05174831, -0.00118444, ...,  0.02656641,
         -0.06316595, -0.00989179],
        [ 0.06455711, -0.09597485, -0.00802086, ...,  0.04556075,
          0.03425689, -0.06320321],
        [ 0.03737191,  0.01609945, -0.01188287, ...,  0.01971767,
          0.03197535,  0.03681566],
        ...,
        [-0.01434366,  0.04050243,  0.03178749, ..., -0.01347778,
          0.01231928, -0.00433748],
        [-0.01434366,  0.04050243,  0.03178749, ..., -0.01347778,
          0.01231928, -0.00433748],
        [-0.01434366,  0.04050243,  0.03178749, ..., -0.01347778,
          0.01231928, -0.00433748]]], dtype=float32)>

In [74]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.03252532, 0.        , 0.        , 0.        , 0.08137994,
         0.05716804, 0.07646164, 0.        , 0.06267914, 0.        ,
         0.        , 0.05527152, 0.03471658, 0.        , 0.06052066,
         0.07369411, 0.03999329, 0.07233908, 0.        , 0.        ,
         0.        , 0.        , 0.00167199, 0.02483571, 0.03850508,
         0.00659957, 0.04933478, 0.        , 0.08017987, 0.00766611,
         0.        , 0.08330828],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.00517504, 0.00881704, 0.0152176 , 0.06772353,
         0.        , 0.05791776, 0.03621373, 0.00522004, 0.06312861,
         0.        , 0.05138377, 0.01683826, 0.        , 0.        ,
         0.04150048, 0.04523654, 0.05826786, 0.        , 0.        ,
         0.        , 0.00661713, 0.03841821, 0.00675183, 0.05907963,
         0.        , 0.        ],
        [0.        , 0.0397848 , 0.01553619, 0.0553

In [75]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.03252532, 0.0397848 , 0.01553619, 0.05530822, 0.08137994,
        0.05716804, 0.07646164, 0.0804579 , 0.06267914, 0.08577524,
        0.02801581, 0.07149342, 0.03890397, 0.02012528, 0.06506567,
        0.07369411, 0.05138377, 0.07630368, 0.0169099 , 0.        ,
        0.04150048, 0.04523654, 0.05826786, 0.02536781, 0.03850508,
        0.01947673, 0.04933478, 0.04512004, 0.08017987, 0.07087858,
        0.01724184, 0.08330828]], dtype=float32)>